# Satria Data Hitam

# Deskripsi Projek

- Untuk keperluan mengefektifkan pengelolaan anggaran BPJS Kesehatan, maka pada lomba BDC Satria Data 2022 Anda ditantang untuk melakukan prediksi status pulang peserta berdasarkan data FKTP BPJS Kesehatan. 
- Bangun pemodelan untuk klasifikasi status pulang peserta menjadi “Sehat” dan “Belum_Sehat” dengan memanfaatkan data train. 


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

In [3]:
col = ['Nomor peserta',
'Nomor keluarga',
'Bobot',
'ID Kunjungan FKTP',
'Tanggal datang kunjungan FKTP',
'Tanggal pulang kunjungan FKTP',
'Provinsi FKTP',
'Kabupaten/Kota FKTP',
'Kepemilikan FKTP',
'Jenis FKTP',
'Tipe FKTP',
'Tingkat Pelayanan FKTP',
'Jenis Poli FKTP',
'Segmen Peserta saat akses layanan FKTP',
'Kode dan nama diagnosis ICD 10 (3 digit)',
'Kode diagnosis ICD 10 (3 digit)',
'Kode diagnosis (beragam 3-5 digit)',
'Nama diagnosis berasal dari kode diagnosis FKP15',
'Provinsi faskes tujuan rujukan',
'Kabupaten/Kota faskes tujuan rujukan',
'Kepemilikan faskes tujuan rujukan',
'Jenis faskes tujuan rujukan',
'Tipe faskes tujuan rujukan',
'Poli faskes tujuan rujukan',
'Jenis Kunjungan FKTP',
'Kelas Status Pulang Peserta']

In [4]:
train = pd.read_csv(r'..\Satria-Data-BDC\train_fktp.txt', names=col, infer_datetime_format=True, parse_dates=[4,5])
test = pd.read_csv(r'..\Satria-Data-BDC\test_fktp.txt', names=col[:-1], infer_datetime_format=True, parse_dates=[4,5])

# Feature Engineering

In [5]:
train1 = train.copy()
test1 = test.copy()

clean_data = [train1, test1]

## Exploratory Data Analysis

In [6]:
print(f'Dimensi Train:\n{train.shape}')
print('-'*10)
print(f'Dimensi Test:\n{test.shape}')

Dimensi Train:
(4056898, 26)
----------
Dimensi Test:
(1014225, 25)


In [7]:
train.info(show_counts=True)
print('-'*10)
train.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4056898 entries, 0 to 4056897
Data columns (total 26 columns):
 #   Column                                            Non-Null Count    Dtype         
---  ------                                            --------------    -----         
 0   Nomor peserta                                     4056898 non-null  float64       
 1   Nomor keluarga                                    4056898 non-null  int64         
 2   Bobot                                             4056898 non-null  float64       
 3   ID Kunjungan FKTP                                 4056898 non-null  object        
 4   Tanggal datang kunjungan FKTP                     4056898 non-null  datetime64[ns]
 5   Tanggal pulang kunjungan FKTP                     4056898 non-null  datetime64[ns]
 6   Provinsi FKTP                                     4056898 non-null  int64         
 7   Kabupaten/Kota FKTP                               4056898 non-null  int64         
 8   Ke

,Nomor peserta,Nomor keluarga,Bobot,ID Kunjungan FKTP,Tanggal datang kunjungan FKTP,Tanggal pulang kunjungan FKTP,Provinsi FKTP,Kabupaten/Kota FKTP,Kepemilikan FKTP,Jenis FKTP,Tipe FKTP,Tingkat Pelayanan FKTP,Jenis Poli FKTP,Segmen Peserta saat akses layanan FKTP,Kode dan nama diagnosis ICD 10 (3 digit),Kode diagnosis ICD 10 (3 digit),Kode diagnosis (beragam 3-5 digit),Nama diagnosis berasal dari kode diagnosis FKP15,Provinsi faskes tujuan rujukan,Kabupaten/Kota faskes tujuan rujukan,Kepemilikan faskes tujuan rujukan,Jenis faskes tujuan rujukan,Tipe faskes tujuan rujukan,Poli faskes tujuan rujukan,Jenis Kunjungan FKTP,Kelas Status Pulang Peserta
737968,71486604.0,71486604,135.202271,782348178,2019-02-21,2019-02-21,13,1312,3,1,4,1,13.0,2,9999,NaN,9999,9999,98,9998,98,98,98,98,2,Sehat
1472673,15216666.0,46400922,93.472412,854362327,2019-05-03,2019-05-03,33,3303,3,1,3,1,12.0,4,9999,NaN,9999,9999,98,9998,98,98,98,98,2,Sehat
1459480,212992976.0,212992547,100.214180,307200419P000139,2019-04-06,2019-04-06,73,7309,3,1,4,1,4.0,2,1742,Z09,Z090,Follow-up examination after surgery for other ...,98,9998,98,98,98,98,1,Belum_Sehat


In [8]:
print(f'Jumlah sample duplicated dalam Train:\n{train1.duplicated().sum()}')
print('-'*10)
print(f'Jumlah sample duplicated dalam Test:\n{test1.duplicated().sum()}')

Jumlah sample duplicated dalam Train:
0
----------
Jumlah sample duplicated dalam Test:
0


In [9]:
print(f'Jumlah null values pada setiap kolom di Train:\n{train1.isnull().sum()}')
print('-'*10)
print(f'Jumlah null values pada setiap kolom di Test:\n{test.isnull().sum()}')

Jumlah null values pada setiap kolom di Train:
Nomor peserta                                             0
Nomor keluarga                                            0
Bobot                                                     0
ID Kunjungan FKTP                                         0
Tanggal datang kunjungan FKTP                             0
Tanggal pulang kunjungan FKTP                             0
Provinsi FKTP                                             0
Kabupaten/Kota FKTP                                       0
Kepemilikan FKTP                                          0
Jenis FKTP                                                0
Tipe FKTP                                                 0
Tingkat Pelayanan FKTP                                    0
Jenis Poli FKTP                                           1
Segmen Peserta saat akses layanan FKTP                    0
Kode dan nama diagnosis ICD 10 (3 digit)                  0
Kode diagnosis ICD 10 (3 digit)                     1

In [10]:
for i in col[:-1]:
    print(f'Jumlah unique value kolom {i} pada Test: {len(test[i].unique())}')

Jumlah unique value kolom Nomor peserta pada Test: 478035
Jumlah unique value kolom Nomor keluarga pada Test: 298556
Jumlah unique value kolom Bobot pada Test: 37316
Jumlah unique value kolom ID Kunjungan FKTP pada Test: 1014225
Jumlah unique value kolom Tanggal datang kunjungan FKTP pada Test: 731
Jumlah unique value kolom Tanggal pulang kunjungan FKTP pada Test: 735
Jumlah unique value kolom Provinsi FKTP pada Test: 34
Jumlah unique value kolom Kabupaten/Kota FKTP pada Test: 505
Jumlah unique value kolom Kepemilikan FKTP pada Test: 9
Jumlah unique value kolom Jenis FKTP pada Test: 6
Jumlah unique value kolom Tipe FKTP pada Test: 9
Jumlah unique value kolom Tingkat Pelayanan FKTP pada Test: 3
Jumlah unique value kolom Jenis Poli FKTP pada Test: 26
Jumlah unique value kolom Segmen Peserta saat akses layanan FKTP pada Test: 5
Jumlah unique value kolom Kode dan nama diagnosis ICD 10 (3 digit) pada Test: 1663
Jumlah unique value kolom Kode diagnosis ICD 10 (3 digit) pada Test: 1663
Jumlah

## Data Cleaning

In [12]:
for dataset in clean_data:
    drop_col = ['Nomor peserta', 'Nomor keluarga', 'Bobot', 'ID Kunjungan FKTP', 'Kode dan nama diagnosis ICD 10 (3 digit)', 'Kode diagnosis (beragam 3-5 digit)', 'Nama diagnosis berasal dari kode diagnosis FKP15']
    dataset.drop(drop_col, axis=1, inplace=True)

    dataset.dropna()

## Feature Engineering

In [13]:
for dataset in clean_data:
    dataset['Lama kunjungan FKTP'] = dataset['Tanggal pulang kunjungan FKTP']-dataset['Tanggal datang kunjungan FKTP']

In [14]:
train1

,Tanggal datang kunjungan FKTP,Tanggal pulang kunjungan FKTP,Provinsi FKTP,Kabupaten/Kota FKTP,Kepemilikan FKTP,Jenis FKTP,Tipe FKTP,Tingkat Pelayanan FKTP,Jenis Poli FKTP,Segmen Peserta saat akses layanan FKTP,Kode diagnosis ICD 10 (3 digit),Provinsi faskes tujuan rujukan,Kabupaten/Kota faskes tujuan rujukan,Kepemilikan faskes tujuan rujukan,Jenis faskes tujuan rujukan,Tipe faskes tujuan rujukan,Poli faskes tujuan rujukan,Jenis Kunjungan FKTP,Kelas Status Pulang Peserta,Lama kunjungan FKTP
0,2019-08-01,2019-08-01,61,6108,3,1,4,1,12.0,2,NaN,98,9998,98,98,98,98,2,Sehat,0 days
1,2020-06-17,2020-06-17,51,5171,9,3,2,1,1.0,4,K29,98,9998,98,98,98,98,1,Belum_Sehat,0 days
2,2019-06-01,2019-06-01,35,3516,9,2,1,1,3.0,4,Z30,98,9998,98,98,98,98,1,Belum_Sehat,0 days
3,2019-12-21,2019-12-21,34,3402,3,1,4,1,1.0,5,I10,98,9998,98,98,98,98,1,Belum_Sehat,0 days
4,2019-11-04,2019-11-04,35,3509,3,1,4,1,1.0,1,I11,98,9998,98,98,98,98,1,Belum_Sehat,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4056893,2020-07-29,2020-07-29,35,3505,9,3,2,1,12.0,4,NaN,98,9998,98,98,98,98,2,Sehat,0 days
4056894,2020-11-14,2020-11-14,35,3514,9,2,1,1,1.0,5,H26,35,3514,9,2,14,99,1,Belum_Sehat,0 days
4056895,2019-08-16,2019-08-16,11,1171,4,2,1,1,13.0,5,NaN,98,9998,98,98,98,98,2,Sehat,0 days
4056896,2020-04-01,2020-04-01,74,7402,3,1,3,1,1.0,2,K30,98,9998,98,98,98,98,1,Belum_Sehat,0 days
